# 0 準備

## 0.1 ライブラリのインストール

In [ ]:
# リポジトリ
!git clone https://github.com/nu-dialogue/clip-prefix-caption-jp.git
%cd clip-prefix-caption-jp


# 必要ライブラリインストール
!pip install git+https://github.com/openai/CLIP.git
!pip install git+https://github.com/rinnakk/japanese-clip.git
!pip install scikit-image torch transformers sentencepiece

## 0.2 ライブラリのインポート

In [ ]:
import json
import os
import sys
import random
import torch
import argparse
from IPython.display import display
from google.colab import files
from huggingface_hub import hf_hub_download

from preprocess import prepare_data
from inference import Predictor
from model import build_models_from_pretrained
from train import set_default_args_to_parser, train

# 後で使う関数も作っておく
def upload_file():
  uploaded = files.upload()
  if not uploaded:
    image_fpath = ''
  elif len(uploaded) == 1:
    image_fpath = list(uploaded.keys())[0]
  else:
    raise RuntimeError("1度に1枚まで")
  return image_fpath

# 1 自分のデータセットを用意

## 1.1 データの設置

data/ディレクトリに以下のデータを用意
```
data/
  └original/ # データセット名
    ├images/ # 画像データを含んだフォルダ
    │  ├001.jpeg # 画像ファイル名は何でもよい（連番である必要はない）
    │  ├002.jpeg
    │  └...
    │
    └captions.csv # 画像ファイル名とそのキャプション文のペアリスト
```

captions.csvの中身
```csv
001.jpeg,スケボーに興じる一人の男性がいます。
002.jpeg,ゲレンデでスキーをしている人がいます。
...
```

## 1.2 データの前処理
1.1で用意したデータセットについて，以下の処理を施す
- CLIPモデルを使用して各画像を埋め込み表現に変換しておく
- 全事例をtraining/validation/test に分割する

In [ ]:
prepare_data(clip_model_name="en_clip_b32",
             captions_fpath="data/original/captions.csv",
             image_dpath="data/original/images",
             test_ratio=0.1,
             valid_ratio=0.1,
             train_ratio=0.8,
             shuffle=True)

# 2 自分が用意したデータでfine-tuning

## 2.1 COCOで事前学習済みのモデルをダウンロード
COCOデータセットで事前に学習された重みでモデルを初期化する

In [ ]:
pretrained_repo_id = "nu-dialogue/sfcoco2022-clipcap"
_ = hf_hub_download(repo_id=pretrained_repo_id, filename="coco-gpt_medium-en_clip_b32-transformer-finetune-ep10-bs48-lr2e-05/args.json")
coco_checkpoint_fpath = hf_hub_download(repo_id=pretrained_repo_id, filename="coco-gpt_medium-en_clip_b32-transformer-finetune-ep10-bs48-lr2e-05/004.pt")

## 2.2 学習
学習パラメータのtips
- (TODO)

In [ ]:
# 学習の引数を設定
parser = argparse.ArgumentParser()
set_default_args_to_parser(parser)
args = parser.parse_args(args=[
    '--train_name_prefix', "coco_based",
    '--dataset_name', 'original',
    '--rinna_gpt_name', 'gpt_medium',
    '--clip_model_name', 'en_clip_b32',
    '--pretrained_path', coco_checkpoint_fpath,
    '--epochs', '20',
    '--per_gpu_train_batch_size', '8',
    '--save_every', '2',
    '--n_gpu', '1'
    # その他パラメータはset_default_args_to_parser()を参照せよ
])

# 学習実行
# ベストモデルのパスがbest_pt_fpathに格納される
_, best_pt_fpath = train(args=args)

## 2.3 Fine-tunedモデルの読み込み

In [ ]:
cap_model, cap_tokenizer, clip_model, clip_preprocess = build_models_from_pretrained(best_pt_fpath)
original_predictor = Predictor(cap_model=cap_model, cap_tokenizer=cap_tokenizer,
                               clip_model=clip_model, clip_preprocess=clip_preprocess)

## 2.4 テスト画像の選択

In [ ]:
#@markdown ### 好きな画像をアップロードする場合
#@markdown ローカルにある画像を使いたい場合は，このセルを実行してアップロードしてください．
#@markdown アップロードした画像はカレントディレクトリ直下に吐き出されます．

image_fpath = upload_file()

In [ ]:
#@markdown ### テスト画像リストから選ぶ場合
#@markdown テスト画像リストの画像を使用する場合は，このセルを実行して1枚選択してください．

# テスト画像ファイルリスト読込
TEST_IMAGE_FNAME_LIST = json.load(open("data/original/processed-en_clip_b32/test_list.json"))

# 1枚選択
image_fname = TEST_IMAGE_FNAME_LIST[0]
# image_fname = random.choice(TEST_IMAGE_FNAME_LIST)

image_fpath = os.path.join("data/original/images", image_fname)

## 2.5 キャプション生成

In [ ]:
# キャプション生成
pil_image, captions = original_predictor.caption(image_fpath=image_fpath, beam_size=5)
display(pil_image) # 画像を表示
print(*captions, sep="\n") # キャプションを表示